<a href="https://colab.research.google.com/github/vmt311/Finetune-LLM/blob/main/FinetuneLLM_guessPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Python >=3.10
!pip install -U "transformers>=4.44" "datasets>=2.20" "accelerate>=0.33" "peft>=0.13" \
  "trl>=0.9" "bitsandbytes>=0.43" "evaluate" "scipy" "sentencepiece"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.10.0
    Uninstalling accelerate-1.10.0:
      Successfully uninstalled accelerate-1.10.0
  Attempting uninstall: peft
    Found existing installation: peft 0.17.0
    Uninstalling

In [ ]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get("HF_TOKEN")

login(token=hf_token)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("vmt311/bookpricer-data")

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'price'],
        num_rows: 260974
    })
    test: Dataset({
        features: ['text', 'price'],
        num_rows: 28998
    })
})

In [ ]:
train=dataset["train"]
test=dataset["test"]

In [ ]:
train[1]

{'text': "How much does this book cost to the nearest dollar?\n\nHell's Kitchen Cookbook\nWelcome to the Hell's Kitchen Cookbook, and to the culinary expertise of Gary Rhodes and Jean-Christophe Novelli from TV's hottest reality show. Tantalised by the delicious recipes? Fascinated by the tears, tempers and tantrums of the ktchen? Watched mouth watereing as novices were beaten, whipped and pressure cooked into cordon-bleu-standard chefs in a matter of weeks? Now the Hell's Kitchen Cookbook brings the inferno right into your own home - A selection of stunning recipes from Gary Rhodes and Jean-Christophe Novelli adapted to use at home. - The contestants' signiture dishes so you can serve up your very own King Prawn Rockefella courtesy of Terry. - The highs and lows of the show rel\n\nPrice is $54.00",
 'price': 53.97}

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    HfArgumentParser, TrainingArguments, pipeline, logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
base_model_name = "meta-llama/Meta-Llama-3.1-8B"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
base_model

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_size = "right"

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
# Cấu hình LoRA
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
sft_config = SFTConfig(
    output_dir="./sft-model-book-price",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    gradient_checkpointing=False,
    save_strategy="no",

    # riêng cho SFTTrainer
    # max_seq_length=1024,
    packing=False,
)

In [ ]:
def formatting_func(example):
    return f"User: {example['text']}\nAssistant: {example['price']}"

In [ ]:
formatting_func(train[0])

"User: How much does this book cost to the nearest dollar?\n\nExtremely Weird Animal Defenses\nDescribes the physical characteristics and behavior of a number of animals with unusual means of defense, including the burrfish, the gelada, and the bulldog ant Publisher Rick Steves (April 16, 1997), Language English, Paperback 32 pages, ISBN 10 ISBN 13 Reading age 8 - 12 years, Grade level 3 - 7, Item Weight 4.8 ounces, Dimensions 8.5 x 0.25 x 11.5 inches Categories Books, Science & Math, Biological Sciences Author 'avatar' 'https 'name' 'Sarah Lovett', 'about' 'Discover more of the author’s books, see similar authors, read author blogs and more'\n\nPrice is $3.00\nAssistant: 3.2"

In [ ]:
trainer = SFTTrainer(
    model=base_model,
    processing_class=tokenizer,
    train_dataset=train,
    peft_config=peft_config,
    formatting_func=formatting_func,
    args=sft_config,
    eval_dataset=test
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/260974 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/260974 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
25,2.722100
50,2.639700
75,2.248300
100,2.096600
125,2.027900
150,2.025400
175,1.982800
200,1.946700
225,1.974800
250,1.930800


In [ ]:
trainer.save_model()

In [ ]:
trainer.push_to_hub(
    model_name="vmt311/sft-model-book-price"
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /content/results/training_args.bin    : 100%|##########| 6.10kB / 6.10kB            

  ...vents.1756196333.7629f0683cec.561.0: 100%|##########|  344kB /  344kB            

  ...t/results/adapter_model.safetensors:  38%|###8      | 41.9MB /  109MB            

  /content/results/tokenizer.json       : 100%|##########| 17.2MB / 17.2MB            

CommitInfo(commit_url='https://huggingface.co/vmt311/results/commit/7710fb485777e1ff3e8ac76e541d091ab3efa3c7', commit_message='End of training', commit_description='', oid='7710fb485777e1ff3e8ac76e541d091ab3efa3c7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/vmt311/results', endpoint='https://huggingface.co', repo_type='model', repo_id='vmt311/results'), pr_revision=None, pr_num=None)

In [ ]:
!nvidia-smi

Tue Aug 26 13:06:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             49W /  400W |   12255MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----